In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
files = sorted(glob('output/*.json'), key = lambda x: int(x.split('-')[1].replace('.json', '')))
len(files)

42729

In [4]:
!head -n 1 pseudolabel.jsonl

{"predict_ms": "<|startoftranscript|><|ms|><|transcribe|> anda tahu keuntungan boleh lebih tinggi daripada keuntungan kewangan rumah maka saya tidak akan mencari dalam akaun saya akan mencari ke dalam ethereum atau beberapa crypto punks bergantung pada faktor risiko anda kerana rumah kajang dihantar tidak mengganggu dsr saya sejauh ini jadi sekarang apa posisi saya untuk mendapatkan kewangan ketiga jadi mungkin setelah melihat sekeliling saya menemui seorang penjual yang dapat menutupi perhubungan tetapi bank hanya menerima 70% dari itu saya boleh membayar perbezaan dengan menggunakan wang ini kerana sekali lagi ia menyusahkan saya dan aset tetapi jika anda tidak selesa dengan mencari<|endoftext|>", "predict_en": "<|startoftranscript|><|en|><|transcribe|> you know the returns can be higher than the savings of the housing loan interest then i will not put in the account i'll put into ethereum or some crypto punks depending on your risk factor then because of the kajang house being let o

In [5]:
import mp
import copy

minimum_score = 6

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = os.path.join('output-audio', f'{f_split}-{i}.mp3')
            if not os.path.exists(audio_filename):
                continue
            
            if data[i]['score_ms'] > minimum_score:
                t = data[i]['predict_ms']
                a = np.array(tokenizer.encode(t, add_special_tokens = False))
                a = a[a != 50257]
                if len(a) > 5 and len(a) + 1 < maxlen:
                    dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t]).todense()
                    repeat = (dense > 3).sum() > 1
                    if not repeat:
                        new_text = tokenizer.decode(a.tolist() + [50257])
                        results.append({
                            'new_text': new_text,
                            'audio_filename': audio_filename
                        })
            
            if data[i]['score_en'] > minimum_score:
                t = data[i]['predict_en']
                a = np.array(tokenizer.encode(t, add_special_tokens = False))
                a = a[a != 50257]
                if len(a) > 5 and len(a) + 1 < maxlen:
                    dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t]).todense()
                    repeat = (dense > 3).sum() > 1
                    if not repeat:
                        new_text = tokenizer.decode(a.tolist() + [50257])
                        results.append({
                            'new_text': new_text,
                            'audio_filename': audio_filename
                        })

    return results

In [6]:
results = mp.multiprocessing(files, loop, cores = 30)

100%|██████████| 1424/1424 [02:14<00:00, 10.62it/s]


In [7]:
len(results)

2333268

In [9]:
results[4]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|> dan itu boleh menjadi sangat langsung. Tetapi yang paling menakutkan adalah apabila banyak orang menghadap jumlah wang yang besar ini untuk kali pertama yang sangat-sangat, ini adalah ketika literasi kewangan datang kerana jumlah wang itu, tiba-tiba semuanya menjadi jip kerana anda melihat semuanya betul, ketika anda tahu bahawa anda mempunyai sejuta dalam bank, ya, tidak mengapa, ya, tidak mengapa, tidak mengapa, tidak mengapa, dan secara langsung anda memperbaiki gaya hidup anda tanpa<|endoftext|>',
 'audio_filename': 'output-audio/1-0-2.mp3'}

In [10]:
with open('prepared-pseudolabel.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 2333268/2333268 [00:10<00:00, 230429.61it/s]


In [11]:
from streaming import MDSWriter, LocalDataset

In [12]:
columns = {
    'new_text': 'str',
    'audio_filename': 'str'
}

hashes = 'sha1', 'xxh64'

In [13]:
!rm -rf mosaic-pseudolabel

In [14]:
with MDSWriter(out='mosaic-pseudolabel', columns=columns, compression=None, hashes=hashes) as out:
    with open('prepared-pseudolabel.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            sample = {
                'new_text': l['new_text'],
                'audio_filename': l['audio_filename'],
            }
            out.write(sample)

2333268it [00:17, 135549.21it/s]


In [19]:
dataset = LocalDataset('mosaic-pseudolabel')
len(dataset)

2333268

In [16]:
dataset[0]

{'audio_filename': 'output-audio/1-0-0.mp3',
 'new_text': '<|startoftranscript|><|ms|><|transcribe|> anda tahu keuntungan boleh lebih tinggi daripada keuntungan kewangan rumah maka saya tidak akan mencari dalam akaun saya akan mencari ke dalam ethereum atau beberapa crypto punks bergantung pada faktor risiko anda kerana rumah kajang dihantar tidak mengganggu dsr saya sejauh ini jadi sekarang apa posisi saya untuk mendapatkan kewangan ketiga jadi mungkin setelah melihat sekeliling saya menemui seorang penjual yang dapat menutupi perhubungan tetapi bank hanya menerima 70% dari itu saya boleh membayar perbezaan dengan menggunakan wang ini kerana sekali lagi ia menyusahkan saya dan aset tetapi jika anda tidak selesa dengan mencari<|endoftext|>'}

In [17]:
!git clone https://huggingface.co/datasets/malaysia-ai/mosaic-pseudolabel-malaysian-youtube-whisper-large-v3
!cp mosaic-pseudolabel/* mosaic-pseudolabel-malaysian-youtube-whisper-large-v3

fatal: destination path 'mosaic-pseudolabel-malaysian-youtube-whisper-large-v3' already exists and is not an empty directory.


In [ ]:
!rm -rf mosaic-pseudolabel

In [18]:
dataset = LocalDataset('mosaic-pseudolabel-malaysian-youtube-whisper-large-v3')
len(dataset)

2333268

In [20]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj='prepared-pseudolabel.jsonl',
    path_in_repo='prepared-pseudolabel.jsonl',
    repo_id='mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3',
    repo_type='dataset',
)

prepared-pseudolabel.jsonl:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3/blob/main/prepared-pseudolabel.jsonl'